In [21]:
%pip install pandas < 2.0.0
%pip install numpy scikit-learn pygrinder benchpots pypots

/bin/bash: line 1: 2.0.0: No such file or directory
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
# Importa bibliotecas necessárias para manipulação de dados e pré-processamento
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar, calc_missing_rate
from benchpots.datasets import preprocess_physionet2012

# Carrega e pré-processa o conjunto de dados PhysioNet 2012
# subset='set-a' seleciona o subconjunto A do dataset
# rate=0.1 adiciona artificialmente 10% de valores faltantes aos dados para teste
data = preprocess_physionet2012(subset='set-a', rate=0.1)

# Separa os dados em conjuntos de treino, validação e teste
# Cada conjunto contém séries temporais com valores faltantes (originais + artificiais)
train_X, val_X, test_X = data["train_X"], data["val_X"], data["test_X"]

# Exibe as dimensões dos dados: (n_amostras, n_passos_temporais, n_características)
print(train_X.shape)
print(val_X.shape)  # O número de amostras varia entre treino e validação, mas passos e características são iguais

# Calcula e exibe a taxa de valores faltantes no conjunto de treino
print(f"Temos {calc_missing_rate(train_X):.1%} valores faltantes em train_X")

# Prepara o conjunto de treino: apenas as séries temporais incompletas são necessárias
train_set = {"X": train_X}

# Prepara o conjunto de validação: inclui tanto os dados incompletos quanto os originais completos
# X_ori serve como ground truth para avaliar o desempenho do modelo e selecionar o melhor checkpoint
val_set = {
    "X": val_X,
    "X_ori": data["val_X_ori"],
}

# Prepara o conjunto de teste: contém apenas as séries temporais incompletas
# O modelo irá imputar (preencher) os valores faltantes neste conjunto
test_set = {"X": test_X}

# Obtém os valores originais completos do conjunto de teste para avaliação final
test_X_ori = data["test_X_ori"]

# Cria uma máscara que identifica os valores que foram artificialmente removidos
# Esta máscara indica onde estão os valores que existem em X_ori mas foram removidos em test_X
# Usa XOR (^) para encontrar posições que são faltantes em test_X mas presentes em test_X_ori
indicating_mask = np.isnan(test_X) ^ np.isnan(test_X_ori)

# Importa o modelo SAITS (Self-Attention-based Imputation for Time Series)
from pypots.imputation import SAITS
from pypots.nn.functional import calc_mae

# Configura e inicializa o modelo SAITS com os hiperparâmetros:
# - n_steps: número de passos temporais na série
# - n_features: número de características/variáveis em cada passo
# - n_layers: 2 camadas de transformers
# - d_model: 256 dimensões no espaço de embedding
# - n_heads: 4 cabeças de atenção paralelas
# - d_k, d_v: 64 dimensões para keys e values no mecanismo de atenção
# - d_ffn: 128 dimensões na rede feed-forward
# - dropout: 0.1 taxa de dropout para regularização
# - epochs: 5 épocas de treinamento
saits = SAITS(
    n_steps=train_X.shape[1],
    n_features=train_X.shape[2],
    n_layers=2,
    d_model=256,
    n_heads=4,
    d_k=64,
    d_v=64,
    d_ffn=128,
    dropout=0.1,
    epochs=5
)

# Treina o modelo usando o conjunto de treino e valida com o conjunto de validação
# O modelo aprende a reconstruir valores faltantes através de mecanismos de auto-atenção
saits.fit(train_set, val_set)

# Usa o modelo treinado para imputar (preencher) todos os valores faltantes no conjunto de teste
# Isso inclui tanto os valores originalmente faltantes quanto os artificialmente removidos
imputation = saits.impute(test_set)

# Calcula o erro absoluto médio (MAE) comparando os valores imputados com os valores verdadeiros
# Avalia apenas os valores que foram artificialmente removidos (indicados pela máscara)
mae = calc_mae(imputation, np.nan_to_num(test_X_ori), indicating_mask)

print(f"MAE: {mae:.4f}")

# Salva o modelo treinado em arquivo para uso futuro
# overwrite=True permite sobrescrever o arquivo se ele já existir
saits.save("save_it_here/saits_physionet2012.pypots", overwrite=True)

# Recarrega o modelo salvo do arquivo
# Útil para continuar o treinamento ou realizar novas imputações posteriormente
saits.load("save_it_here/saits_physionet2012.pypots")


2026-02-10 16:40:12 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2026-02-10 16:40:12 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2026-02-10 16:40:12 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2026-02-10 16:40:12 [INFO]: Loaded successfully!
2026-02-10 16:40:21 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2026-02-10 16:40:21 [INFO]: 22718 values masked out in the val set as ground truth, take 10.01% of the original observed values
2026-02-10 16:40:21 [INFO]: 29258 values masked out in the test set as ground truth, take 10.06% of the original observed values
2026-02-10 16:40:21 [INFO]: Total sample number: 3997
2026-02-10 16:40:21 [I

(2557, 48, 37)
(640, 48, 37)
Temos 79.7% valores faltantes em train_X


2026-02-10 16:40:27 [INFO]: Epoch 001 - training loss (MAE): 0.9659, validation MSE: 0.4094
2026-02-10 16:40:25 [INFO]: Epoch 002 - training loss (MAE): 0.6655, validation MSE: 0.3597
2026-02-10 16:40:38 [INFO]: Epoch 003 - training loss (MAE): 0.5968, validation MSE: 0.3531
2026-02-10 16:40:44 [INFO]: Epoch 004 - training loss (MAE): 0.5615, validation MSE: 0.3418
2026-02-10 16:40:50 [INFO]: Epoch 005 - training loss (MAE): 0.5400, validation MSE: 0.3337
2026-02-10 16:40:50 [INFO]: Finished training. The best model is from epoch#5.
2026-02-10 16:40:50 [WARNING]: ‼️ File save_it_here/saits_physionet2012.pypots exists. Argument `overwrite` is True. Overwriting now...
2026-02-10 16:40:50 [INFO]: Saved the model to save_it_here/saits_physionet2012.pypots
2026-02-10 16:40:50 [INFO]: Model loaded successfully from save_it_here/saits_physionet2012.pypots


MAE: 0.3194
